In [ ]:
!pip install objectnat iduedu

In [ ]:
%load_ext autoreload
%autoreload 2
from iduedu import get_intermodal_graph, get_boundary

poly = get_boundary(osm_id=1114252)
G_intermodal = get_intermodal_graph(polygon=poly, clip_by_bounds=True)
# Retrieving the intermodal graph for a specific region using OSM ID 1114252. This graph includes transportation networks
# (such as roads, public transport routes) for a given area.

In [ ]:

from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrones

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones

# Point(30.295606, 59.9439234) - Extra point for multiple isochrones

points = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.295606 and latitude 59.9439234).

isochrones_r, stops, routes = get_accessibility_isochrones(
    isochrone_type='radius',
    points=points,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal
)

# Visualization
m = isochrones_r.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')

In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrones

# - 'get_accessibility_isochrones' from 'objectnat' to work with isochrones
# - 'geopandas' for handling geospatial data
# Point(30.295606, 59.9439234) - Extra point for multiple isochrones

points = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).
isochrones_w, stops, routes = get_accessibility_isochrones(
    isochrone_type='ways',
    points=points,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal
)

# Visualization
m = isochrones_w.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')

In [ ]:
m = isochrones_r.explore(tiles='CartoDB positron')
isochrones_w.explore(m=m, color='red')

In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrones' from 'objectnat' to work with stepped isochrones

# Point(30.295606, 59.9439234),Point(30.27060176,59.93546846)

point = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='separate',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=2
)
result.explore(tiles='CartoDB positron', column='dist')


In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrone_stepped' from 'objectnat' to work with stepped isochrones

point = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='radius',
    point=point,
    weight_type="time_min",
    weight_value=10,
    nx_graph=G_intermodal,
    step=2
)
result.explore(tiles='CartoDB positron', column='dist')


In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

# - 'get_accessibility_isochrone_stepped' from 'objectnat' to work with stepped isochrones

point = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326)
# Creating a GeoDataFrame containing a point of interest (with longitude 30.27060176 and latitude 59.93546846).


result = get_accessibility_isochrone_stepped(
    isochrone_type='ways',
    point=point,
    weight_type="time_min",
    weight_value=15,
    nx_graph=G_intermodal,
    step=3
)
result.explore(tiles='CartoDB positron', column='dist')

In [ ]:
from objectnat.methods.utils.graph_utils import graph_to_gdf
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from shapely import Point
import geopandas as gpd
from objectnat import get_accessibility_isochrone_stepped

edges = graph_to_gdf(G_intermodal, nodes=False)
point = gpd.GeoDataFrame(geometry=[Point(30.27060176, 59.93546846)], crs=4326).to_crs(edges.crs)
bbox = gpd.GeoDataFrame(geometry=[poly], crs=4326).to_crs(edges.crs)

type_colors = {
    'walk': '#a3a3a3',        # светло-серый (почти не заметен)
    'bus': '#1f77b4',         # насыщенно-синий
    'trolleybus': '#2ca02c',  # зелёный
    'tram': '#ff7f0e',        # оранжевый
    'subway': '#9467bd',      # фиолетовый
    'boarding': '#8c564b'     # коричневато-серый (можно как нейтральный терминал)
}

edges['color'] = edges['type'].map(type_colors)


steps = [0.1, 0.5, 1, 2, 3, 4, 5]
# steps = [0.1, 0.5]


fig, ax = plt.subplots(figsize=(10, 8), dpi=150)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

edges_plot = edges.plot(ax=ax,color=edges['color'], alpha=0.5, linewidth=0.1, legend=True)
bbox.boundary.plot(ax=ax, color='black', linestyle='--', linewidth=1)
point.plot(ax=ax, color='red', markersize=50)
ax.set_axis_off()

def update(step):
    for coll in ax.collections:
        if coll.get_label() == 'isochrone':
            coll.remove()
    
    result = get_accessibility_isochrone_stepped(
        isochrone_type='separate',
        point=point,
        weight_type="time_min",
        weight_value=15,
        nx_graph=G_intermodal,
        step=step
    )
    result.plot(ax=ax, alpha=1, label='isochrone', column='dist', legend=False)
    ax.set_title(f'Isochrone with step = {step} minutes')

ani = FuncAnimation(
    fig,
    update,
    frames=steps,
    repeat=True,
    interval=2000
)

ani.save('isochrone_animation.gif', writer='pillow', fps=1)